# Baselines

In [4]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/private/home/pfz


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from videoseal.utils.display import save_vid
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, psnr
from videoseal.data.datasets import VideoDataset
from videoseal.augmentation import Identity, H264

import os
from tqdm import tqdm
import torch
import gc

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu" 

In [ ]:
from videoseal.models.baselines import build_baseline

# Directory containing videos
video_dir = "/checkpoint/pfz/projects/videoseal/assets/videos/metamoviegen/"
num_vids = 1
fps = 24 // 1
# frames_per_clip = fps * 3  # 3s
# frame_step = 1

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
# for method in ["hidden"]:
for method in ["cin"]:
    wam = build_baseline(method)
    wam.eval()

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][:num_vids]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {method}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        timer.start()
        vid, mask = VideoDataset.load_full_video_decord(video_path)
        print(f"loading video {video_path} - took {timer.stop():.2f}s")

        # Watermark embedding
        timer.start()
        outputs = wam.embed(vid, is_video=True)
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs = vid  # b c h w
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # psnr
        psnr_score = psnr(imgs, imgs_w).mean().item()
        print(f"PSNR: {psnr_score:.3f}")

        # save
        timer.start()
        save_vid(imgs, f"{method}_{base_name}.mp4", fps)
        save_vid(imgs_w, f"{method}_{base_name}_wmed.mp4", fps)
        save_vid(diff, f"{method}_{base_name}_wm.mp4", fps)

        # Compute min and max values, reshape, and normalize
        min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        normalized_images = (diff - min_vals) / (max_vals - min_vals)

        # Save the normalized video
        save_vid(normalized_images, f"{method}_{base_name}_wm_normalized.mp4", fps)
        print(f"saving videos - took {timer.stop():.2f}s")

        # Augment video
        print(f"compressing and detecting watermarks")
        for crf in [-1, 23, 40]:
            if crf == -1:
                imgs_aug = imgs_w
            else:
                imgs_aug, _ = H264()(imgs_w, crf=crf)

            # detect
            timer.start()
            outputs = wam.detect(imgs_aug, is_video=True)
            preds = outputs["preds"]
            bit_preds = preds[:, 1:]  # b k ...
            bit_accuracy_ = bit_accuracy(
                bit_preds,
                msgs
            ).nanmean().item()
            print(f"CRF={crf} Bit Accuracy: {bit_accuracy_:.3f} - detection took {timer.stop():.2f}s")

        del vid, mask, outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

    # Free model from GPU
    del wam
    torch.cuda.empty_cache()